# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
* Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
* Aggregate the Yelp Data
    * Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    * Aggregate "count" features using this same logic
    * Combine the review document for a restaurant into a CLOB using the same logic
* LEFT JOIN the violations dataset to the Yelp data after aggregationis complete for both datasets

In [1]:
import psycopg2 as psy
import pandas as pd
import re
import numpy as np

In [2]:
#set up connection to our DB
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables"
            " WHERE table_schema='public'" 
            " AND table_type='BASE TABLE'")
rows =cur.fetchall()
print(pd.DataFrame(rows))

                       0
0               business
1                checkin
2             trnt_insps
3  violations_pittsburgh
4                   tips
5                reviews


In [3]:
cur.execute("SELECT * FROM public.restaurants LIMIT 50")
biz = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.checkin LIMIT 50")
checkins = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.reviews LIMIT 50")
reviews = pd.DataFrame(cur.fetchall())

**NOTE: ONLY LIMITING to 50 rows during build phase to limit processing time. **

In [4]:
biz.columns = ['bizID', 'name', 'address', 'zip', 'neighborhood', 'lat','long', 'categories','attributes','is_open','review_count','hours','stars']
biz.describe()

,bizID,name,address,zip,neighborhood,lat,long,categories,attributes,is_open,review_count,hours,stars
count,50,50,50,50,50,50,50,50,50,50,50,50,50
unique,50,49,50,49,29,50,50,49,49,2,29,34,8
top,w9TXFk3zKcVQJ92Ow992mg,LCBO,385 Roncesvalles Avenue,M5G,,43.698757,-79.4143711,"['Restaurants', 'Middle Eastern']",,1,5,,3.5
freq,1,2,1,2,10,1,1,2,2,37,6,16,14


In [5]:
reviews.columns = ['type','cool','bizID','reviewID','userID','stars','text','useful','funny','date']
#get dummies for star rating column
reviews = pd.concat([reviews, pd.get_dummies(reviews['stars'], prefix='stars')], axis=1)
reviews.head()

,type,cool,bizID,reviewID,userID,stars,text,useful,funny,date,stars_1,stars_2,stars_3,stars_4,stars_5
0,b'review',0,b'1AlkWurKaoagQ51Z99pfiA',b'WE6WZprDZLaJLPkoNaxjQA',b'-rFBSjuTYLH6uFRyg4dxXQ',5,"b""I was here Thanksgiving Eve. Given that ever...",0,0,b'2013-12-19',0,0,0,0,1
1,b'review',0,b'1AlkWurKaoagQ51Z99pfiA',b'4KN7darNf8BnF-f8wYzgGg',b'Q9x-MEp8Yx5QG78lU_f2mA',1,b'I love the store it is the Best Juicy Coutur...,1,1,b'2014-04-25',1,0,0,0,0
2,b'review',0,b'1AlkWurKaoagQ51Z99pfiA',b'js2BnfqBnBJxwZLuFG8SJQ',b'LWinqJdSeptUcP2jPvryhw',5,"b""I'm a Juicy Bitch so of course i had to chec...",0,2,b'2011-04-27',0,0,0,0,1
3,b'review',2,b'1AlkWurKaoagQ51Z99pfiA',b'gHWI-XocyWSmiWInPut70A',b'YkMCNI3nVukPL_PIztHZJw',5,"b""Seriously the BEST Juicy store I've been to ...",2,2,b'2013-12-26',0,0,0,0,1
4,b'review',0,b'1AlkWurKaoagQ51Z99pfiA',b'i4x0ZX-iIYBMkC_4lenDTA',b'BKBNuuPT5IndIoa9oC29Nw',4,"b""This is located at the Forum Shops at Caesar...",0,0,b'2011-08-23',0,0,0,1,0


In [6]:
checkins.columns = ['bizID','type','datetime']
checkins.describe()

,bizID,type,datetime
count,50,50,50
unique,50,1,50
top,b'MSFFEZB0T1RQPJU8CfQjNA',b'checkin',"['Mon-15:1', 'Sat-16:1', 'Sat-17:1', 'Thu-17:1..."
freq,1,50,1


In [7]:
#this is to strip the 'b' tags from bizID, delete if done in SQL before import
checkins['bizID'] = checkins['bizID'].str[2:24]
reviews['bizID'] = reviews['bizID'].str[2:24]
biz['bizID'] = biz['bizID'].str[2:24]


In [8]:
#get list of unique biz and create df
#uniquebiz = []
#for x in reviews['bizID']:
    #if x not in uniquebiz:
        #uniquebiz.append(x)

uniquebiz=np.unique(reviews['bizID'])
bizrevs = pd.DataFrame(uniquebiz)
bizrevs.columns = ['bizID']
bizrevs = pd.merge(bizrevs,biz[['bizID','name','address']],on='bizID', how='left')

**Note: merge in cell above won't result in matches for name/address because of 50 row limit put in place. should work after removing limit**

In [9]:
### If possible, we should try to use predefined pandas/numpy packages to do the aggregations below. 
### Looping is very memory intensive and is hanging up with 50K records.

#declare rest of columns
bizrevs['reviews'] = ""
bizrevs['checkins'] = 0
bizrevs['stars_1'] = 0
bizrevs['stars_2'] = 0
bizrevs['stars_3'] = 0
bizrevs['stars_4'] = 0
bizrevs['stars_5'] = 0
bizrevs['reviews_whole_words'] = ""

#look up each unique biz ID in reviews table
for i in range(len(bizrevs['bizID'])):
    #add text of review to reviews column if biz IDs match
    for x in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][x]:
            bizrevs['reviews'][i] = bizrevs['reviews'][i] + reviews['text'][x]
    #count number of checkins
    for y in range(len(checkins['bizID'])):
        if bizrevs['bizID'][i] == checkins['bizID'][y]:
            bizrevs['checkins'][i] = bizrevs['checkins'][i] + 1
    #count number of reviews with each star rating
    for z in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][z]:
            bizrevs['stars_1'][i] = bizrevs['stars_1'][i] + reviews['stars_1'][z]
            bizrevs['stars_2'][i] = bizrevs['stars_2'][i] + reviews['stars_2'][z]
            bizrevs['stars_3'][i] = bizrevs['stars_3'][i] + reviews['stars_3'][z]
            bizrevs['stars_4'][i] = bizrevs['stars_4'][i] + reviews['stars_4'][z]
            bizrevs['stars_5'][i] = bizrevs['stars_5'][i] + reviews['stars_5'][z]
    #extract whole words from reviews
    bizrevs['reviews_whole_words'][i] = ' '.join(re.findall('[A-Za-z]+', bizrevs['reviews'][i]))
    
bizrevs.head()

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

,bizID,name,address,reviews,checkins,stars_1,stars_2,stars_3,stars_4,stars_5,reviews_whole_words
0,1AlkWurKaoagQ51Z99pfiA,NaN,NaN,"b""I was here Thanksgiving Eve. Given that ever...",0,1,0,1,1,3,b I was here Thanksgiving Eve Given that every...
1,3LpJO-rALFVdMEvNygLHPA,NaN,NaN,b'Exceptional massage! Great energy and a comf...,0,0,0,0,0,5,b Exceptional massage Great energy and a comfo...
2,3UCeI_Sm3Q0RcEHsT-WLcQ,NaN,NaN,"b""Saborsito is a great place to grab a cheap, ...",0,0,0,3,2,0,b Saborsito is a great place to grab a cheap t...
3,52fgYGQYyKbJyPlitJt2cg,NaN,NaN,"b""Love this place, the owner is always amazing...",0,0,0,0,0,1,b Love this place the owner is always amazingl...
4,5aYBdm3meMqyKkzJV72Uqw,NaN,NaN,"b""If I could give this company negative stars ...",0,10,2,2,1,2,b If I could give this company negative stars ...


In [10]:
#import toronto inspection data
cur.execute("SELECT * FROM public.trnt_inspn_agg LIMIT 50")
insp = pd.DataFrame(cur.fetchall())
insp.columns = ['insp_bizID','insp_biz_name','insp_biz_address','insp_date','insp_count_minor','insp_count_significant','insp_count_crucial','insp_count_na','insp_total_count_cs']
insp.head()

,insp_bizID,insp_biz_name,insp_biz_address,insp_date,insp_count_minor,insp_count_significant,insp_count_crucial,insp_count_na,insp_total_count_cs
0,10348267,OLLIFFE,1097 YONGE ST,11/30/15,2,0,0,0,0
1,10378852,CORNED BEEF HOUSE,270 ADELAIDE ST W,5/13/15,1,0,0,0,0
2,10377653,BLOOR STREET MARKET,55 BLOOR ST W,1/30/17,1,2,0,0,2
3,10414022,BIRYANI KING,680 REXDALE BLVD,11/17/15,0,0,0,0,0
4,9013311,SAIGON FLOWER RESTAURANT,1138 QUEEN ST W,7/15/15,0,0,0,0,0


In [11]:
#declare function to calculate levenshtein distance between 2 strings (not case sensitive)
def lev(string1, string2):
    #delete the ".lower()" in the following two lines to make distance case sensitive
    s1=str.strip(string1.lower())
    s2=str.strip(string2.lower())
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

#test the function
print(lev('Hello',"hello"))
print(lev('dock','duck '))
print(lev('st','saint'))

0
1
3


**note: next cell should return matches once we include more than the 50 rows (fingers crossed)**

In [12]:
#set value of levenshtein distance threshold (4 means only distances of 3 and lower would be considered)
lev_dist_threshold = 4

#loop through each yelp bizID and find the restaurant with closest lev distance (currently matches using name only)
bizrevs['lev_dist'] = lev_dist_threshold
bizrevs['insp_bizID'] = ""
for i in range(len(bizrevs['bizID'])):
    for x in range(len(insp['insp_bizID'])):
        dist = lev(str(bizrevs['name'][i]),str(insp['insp_biz_name'][x]))
        if dist < bizrevs['lev_dist'][i]:
            bizrevs['lev_dist'][i] = dist
            bizrevs['insp_bizID'][i] = insp['insp_bizID'][x]

df = pd.merge(bizrevs,insp,on='insp_bizID', how='left')            

df.head()

,bizID,name,address,reviews,checkins,stars_1,stars_2,stars_3,stars_4,stars_5,...,lev_dist,insp_bizID,insp_biz_name,insp_biz_address,insp_date,insp_count_minor,insp_count_significant,insp_count_crucial,insp_count_na,insp_total_count_cs
0,1AlkWurKaoagQ51Z99pfiA,NaN,NaN,"b""I was here Thanksgiving Eve. Given that ever...",0,1,0,1,1,3,...,4,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3LpJO-rALFVdMEvNygLHPA,NaN,NaN,b'Exceptional massage! Great energy and a comf...,0,0,0,0,0,5,...,4,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3UCeI_Sm3Q0RcEHsT-WLcQ,NaN,NaN,"b""Saborsito is a great place to grab a cheap, ...",0,0,0,3,2,0,...,4,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52fgYGQYyKbJyPlitJt2cg,NaN,NaN,"b""Love this place, the owner is always amazing...",0,0,0,0,0,1,...,4,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5aYBdm3meMqyKkzJV72Uqw,NaN,NaN,"b""If I could give this company negative stars ...",0,10,2,2,1,2,...,4,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**following section is to normalize addresses. a package is available but doing it manually seems easier / good enough. package:** https://github.com/pnpnpn/street-address 

In [48]:
#normalizes addresses
biz['address'] = [addr.replace('Street','St').replace('Boulevard','Blvd').replace('Avenue','Ave').replace('Road','Rd')
        .replace('North','N').replace('West','W').replace('South','S').replace('East','E') for addr in biz['address']]

#we should also think about removing pre and suffixes, like this example (insp data does not seem to have units etc):
biz['address'][37]

'Liberty Village, 85 Hanna Ave, Unit 103'